# Customer Churn Prediction

In this webinar, we will loads data from the UCI Online Retail data (http://archive.ics.uci.edu/ml/datasets/Online+Retail) and predicts which customers are likely to churn given their purchase activity.

Churn can be defined in many ways. We define churn to be <b>no activity within a period of time (called the churn_period)</b>. Using this definition, a user/customer is said to have churned any form of activity is followed by no activity for an entire duration of time known as the `churn_period` (by default, we assume 30 days). The following figure better illustrates this concept.

<img src="https://dato.com/images/learn/churn-illustration.png"/>

(from our user guide: https://dato.com/learn/userguide/churn_prediction/churn-prediction.html)

We will dig deaper into the different parameters of the Churn Prediction toolkit, but let's start by loading some data!

In [5]:
# Let's import Graphlab Create and a few other libraries
import graphlab as gl
import graphlab.aggregate
import datetime
import time
import os

### Import data from a locally downloaded copy of the UCI data set

Graphlab Create supports loading data from live databases, as well as from local files. In this case, since we're working with a fixed dataset, we will load it from disk.

In [8]:
if  not os.path.exists('Data/online_retail'):
    data = gl.SFrame("https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail.csv")
    data = data.remove_columns(['InvoiceNo', 'Description'])
    data['InvoiceDate']=data['InvoiceDate'].str_to_datetime('%m/%d/%y %H:%M')
    data.save('Data/online_retail')
else:
    data = gl.SFrame('Data/online_retail')

Finally, we want to separate some users into a train/validation set, making sure the validation users are not in the training set, and creating TimeSeries objects out of them.

In [23]:
(train, valid) = gl.churn_predictor.random_split(data, user_id = 'CustomerID', fraction = 0.9, seed = 12)
train_trial = gl.TimeSeries(train, index = 'InvoiceDate')
valid_trial = gl.TimeSeries(valid, index = 'InvoiceDate')

Now we can load user information, which can be used to augment the churn prediction model.

In [24]:
if  not os.path.exists('Data/userdata'):
    userdata = gl.SFrame("https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv")
    userdata.save('Data/userdata')
else:
    userdata = gl.SFrame('Data/userdata')

Downloading https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv to /var/tmp/graphlab-ec2-user/29183/d0e51ef4-9a71-495a-88fc-3b88eb326087.csv

Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv

Parsing completed. Parsed 100 lines in 0.008735 secs.

Finished parsing file https://s3.amazonaws.com/dato-datasets/churn-prediction/online_retail_side_data_extended.csv

Parsing completed. Parsed 4380 lines in 0.006897 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


## Training the model

Let's now train the model.

### Create a train-test split based on users

First, let's observe the data, and see what the time range looks like

In [25]:
print "Start date : %s" % train_trial.min_time
print "End date   : %s" % train_trial.max_time

Start date : 2010-12-01 08:26:00
End date   : 2011-12-09 12:50:00


In [26]:
# Period of inactivity that defines churn -- meaning that if a user stops purchasing
# items for 30 days, we'll consider them as having churned.
churn_period_trial = datetime.timedelta(days = 30) 

# Different beginning of months
churn_boundary_aug = datetime.datetime(year = 2011, month = 8, day = 1) 
churn_boundary_sep = datetime.datetime(year = 2011, month = 9, day = 1) 
churn_boundary_oct = datetime.datetime(year = 2011, month = 10, day = 1) 

In [27]:
model = gl.churn_predictor.create(train_trial,
                                  user_data = userdata,
                                  user_id='CustomerID',
                                  churn_period = churn_period_trial,
                                  time_boundaries = [churn_boundary_aug, churn_boundary_sep, churn_boundary_oct])

PROGRESS: Grouping observation_data by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.

StockCode is a categorical variable with too many different values (4063) and will be ignored.


PROGRESS: Generating features for time-boundary.
PROGRESS: --------------------------------------------------
PROGRESS: Features for 2011-08-01 00:00:00.
PROGRESS: Features for 2011-09-01 00:00:00.
PROGRESS: Features for 2011-10-01 00:00:00.
PROGRESS: Joining user_data with aggregated features.
PROGRESS: --------------------------------------------------
PROGRESS: Training a classifier model.

Boosted trees classifier:

--------------------------------------------------------

...

| 4         | 0.126418     | 0.792410          | 0.503216          |

| 5         | 0.156424     | 0.796868          | 0.489915          |

| 6         | 0.188199     | 0.798826          | 0.480446          |

+-----------+--------------+-------------------+-------------------+


PROGRESS: --------------------------------------------------
PROGRESS: Model training complete: Next steps
PROGRESS: --------------------------------------------------
PROGRESS: (1) Evaluate the model at various timestamps in the past:
PROGRESS:       metrics = model.evaluate(data, time_in_past)
PROGRESS: (2) Make a churn forecast for a timestamp in the future:
PROGRESS:       predictions = model.predict(data, time_in_future)


### Evaluating the model (post-hoc anaylsis)

In [28]:
# Evaluate this model in October
evaluation_time = churn_boundary_oct

In [29]:
metrics = model.evaluate(valid_trial, evaluation_time, user_data = userdata)

PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-10-01 00:00:00
PROGRESS:  End   : 2011-10-31 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.

StockCode is a categorical variable with too many different values (4063) and will be ignored.


PROGRESS: Generating features for boundary 2011-10-01 00:00:00.
PROGRESS: Joining user_data with aggregated features.
PROGRESS: Not enough data to make predictions for 70 user(s). 


In [30]:
print(metrics)

{'auc': 0.696027064538515, 'recall': 0.9465648854961832, 'precision': 0.7492447129909365, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+-----+-----+-----+-----+
| threshold | fpr | tpr |  p  |  n  |
+-----------+-----+-----+-----+-----+
|    0.0    | 1.0 | 1.0 | 262 | 110 |
|   1e-05   | 1.0 | 1.0 | 262 | 110 |
|   2e-05   | 1.0 | 1.0 | 262 | 110 |
|   3e-05   | 1.0 | 1.0 | 262 | 110 |
|   4e-05   | 1.0 | 1.0 | 262 | 110 |
|   5e-05   | 1.0 | 1.0 | 262 | 110 |
|   6e-05   | 1.0 | 1.0 | 262 | 110 |
|   7e-05   | 1.0 | 1.0 | 262 | 110 |
|   8e-05   | 1.0 | 1.0 | 262 | 110 |
|   9e-05   | 1.0 | 1.0 | 262 | 110 |
+-----------+-----+-----+-----+-----+
[100001 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'evaluation_data': Columns:
	CustomerID	int
	probability	float
	label	int

Rows: 372

Data:
+------------+----------------+----

### Make predictions in the future

Here the question to ask is will they churn after a certain period of time. To validate we can see if they user has used us after that evaluation period. Voila! I was confusing it with expiration time (customer churn not usage churn)

In [33]:
# Make predictions in the future.

predictions_trial = model.predict(valid_trial, user_data = userdata)
predictions_trial.print_rows()

PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-12-09 11:20:00
PROGRESS:  End   : 2012-01-08 11:20:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.

StockCode is a categorical variable with too many different values (4063) and will be ignored.


PROGRESS: Generating features for boundary 2011-12-09 11:20:00.
PROGRESS: Joining user_data with aggregated features.
+------------+-----------------+
| CustomerID |   probability   |
+------------+-----------------+
|   13761    |  0.661192655563 |
|   12789    |  0.831865549088 |
|   12377    |  0.929451584816 |
|   13715    |  0.852380812168 |
|   17725    |  0.501834571362 |
|   15437    |  0.89022809267  |
|   12739    |  0.785794794559 |
|   16523    | 0.0530522763729 |
|   14711    |  0.599178552628 |
|   12851    |  0.785794794559 |
+------------+-----------------+
[442 rows x 2 columns]



In [34]:
predictions_trial.sort('probability', ascending=False).print_rows(20,max_column_width=20)

+------------+----------------+
| CustomerID |  probability   |
+------------+----------------+
|   12811    | 0.933722198009 |
|   12377    | 0.929451584816 |
|   12686    | 0.929451584816 |
|   16980    | 0.928296506405 |
|   13863    | 0.926480472088 |
|   15442    | 0.925726234913 |
|   16721    | 0.922811865807 |
|   15083    | 0.922811865807 |
|   12881    | 0.922811865807 |
|   13043    | 0.922811865807 |
|   15303    | 0.922811865807 |
|   14770    | 0.920113921165 |
|   17990    | 0.920113921165 |
|   12866    | 0.914489269257 |
|   16617    | 0.914061486721 |
|   16947    | 0.914061486721 |
|   16957    | 0.914061486721 |
|   14339    | 0.914061486721 |
|   18056    | 0.910906493664 |
|   12945    | 0.909635961056 |
+------------+----------------+
[442 rows x 2 columns]



In [16]:
predictions_trial.sort('probability', ascending=False)[200:300] .print_rows(20,max_column_width=20)

+------------+----------------+
| CustomerID |  probability   |
+------------+----------------+
|   16086    | 0.764054119587 |
|   12990    | 0.760963022709 |
|   12993    | 0.760963022709 |
|   14080    | 0.760963022709 |
|   16596    | 0.760963022709 |
|   16638    | 0.760963022709 |
|   16784    | 0.760963022709 |
|   14987    | 0.760963022709 |
|   15532    | 0.760936200619 |
|   15025    | 0.760936200619 |
|   17772    | 0.760792136192 |
|   17075    | 0.757927417755 |
|   14049    | 0.757856547832 |
|   13972    | 0.757027626038 |
|   15783    | 0.752182364464 |
|   13329    | 0.748927116394 |
|   17636    | 0.74851256609  |
|   16527    | 0.744530022144 |
|   13061    | 0.744530022144 |
|   16122    | 0.740379273891 |
+------------+----------------+
[100 rows x 2 columns]



### Inside the model

In [35]:
model.trained_model

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 9196
Number of feature columns     : 23
Number of unpacked features   : 1290
Number of classes             : 2

Settings
--------
Number of trees               : 10
Max tree depth                : 6
Training time (sec)           : 0.3024
Training accuracy             : 0.8034
Validation accuracy           : None
Training log_loss             : 0.4608
Validation log_loss           : None

In [18]:
model.trained_model.get_feature_importance()

name,index,count
Quantity_features_7,user_user_timesinceseen,73
Age,None,29
Quantity_features_90,sum_sum,12
Quantity_features_60,count_sum,12
Quantity_features_60,sum_ratio,11
UnitPrice_features_7,sum_sum,11
Quantity_features_60,sum_sum,10
Quantity_features_90,sum_firstinteraction_timesinceseen ...,10
UnitPrice_features_90,sum_sum,9
UnitPrice_features_90,sum_max,8
